In [ ]:
from dotenv import load_dotenv
load_dotenv("readonly.env")

In [ ]:
from google.protobuf import message_factory, any_pb2
from sqlalchemy.sql import or_

from couchers.descriptor_pool import get_descriptor_pool
from couchers.db import session_scope
from couchers.models import APICall

from trace_pb2 import Trace

In [ ]:
TYPE_DOMAIN = "type.couchers.org"

In [ ]:
pool = get_descriptor_pool()

factory = message_factory.MessageFactory(pool=pool)

In [ ]:
def get_method_descriptor(method):
    _, service_name, method_name = method.split("/")
    service_descriptor = pool.FindServiceByName(service_name)
    method_descriptor = service_descriptor.FindMethodByName(method_name)
    return method_descriptor

In [ ]:
def get_proto(type_url):
    domain, version, type_name = type_url.split("/")
    return factory.GetPrototype(pool.FindMessageTypeByName(type_name))

In [ ]:
def deserialize_trace(trace: Trace):
    request_proto = get_proto(trace.request.type_url)
    response_proto = get_proto(trace.response.type_url)
    return request_proto.FromString(trace.request.value), response_proto.FromString(trace.response.value) if trace.response.value else None

In [ ]:
traces = []

with session_scope() as session:
    trace_q = (session.query(APICall)
               .filter(APICall.user_id == None)
               #.filter(APICall.method == "/org.couchers.bugs.Bugs/ReportBug")
               #.filter(APICall.time > "2021-06-30T21:03:32.000000+00:00")
               #.filter(APICall.time < "2021-06-30T21:08:55.054000+00:00")
               #.filter(or_(APICall.user_id == user_id, APICall.user_id == None))
               .order_by(APICall.id)
              .limit(100))
    for trace in trace_q.all():
        mdesc = get_method_descriptor(trace.method)
        input_url = f"{TYPE_DOMAIN}/{trace.version}/{mdesc.input_type.full_name}"
        output_url = f"{TYPE_DOMAIN}/{trace.version}/{mdesc.output_type.full_name}"
        trace = Trace(
            id=trace.id,
            version=trace.version,
            time=trace.time.isoformat(),
            method=trace.method,
            status_code=trace.status_code,
            duration=trace.duration,
            user_id=trace.user_id,
            request=any_pb2.Any(type_url=input_url, value=trace.request),
            response=any_pb2.Any(type_url=output_url, value=trace.response),
            traceback=trace.traceback,
        )
        traces.append(trace)

In [ ]:
for trace in traces:
    print(f"*** Call ***")
    print(f"status_code={trace.status_code or ('TB' if trace.traceback else 'OK')}")
    print(f"method={trace.method}")
    print(f"user_id={trace.user_id or 'NA'}")
    print(f"time={trace.time}")
    print(f"duration={trace.duration} ms")
    req, res = deserialize_trace(trace)
    if req:
        print(f"Request: ({trace.request.type_url})")
        print(req)
    if res:
        print(f"Response: ({trace.response.type_url})")
        print(res)
    print()